# *Living Earth* Observed change
# Mapping conversions and modifications of land cover
<img align="right" src="../../Supplementary_data/dea_logo.jpg">

* **[Sign up to the DEA Sandbox](https://app.sandbox.dea.ga.gov.au/)** to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with `DEA Sandbox` environment
* **Products used:** 
[ga_s2am_ard_3](https://explorer.dea.ga.gov.au/products/ga_s2am_ard_3)


## Overview of observed changes

**Land Cover** is the physical and biological cover of the land surface and includes vegetation (managed or semi-natural), water and bare surfaces.  The land cover maps generated through Living Land Management use the legends of the United Nation's Food and Agriculture Organisation (FAO) Land Cover Classification System (LCCS).

Transitions between any two Level 3 land cover types (off-diagonals in a transition matrix) correspond to a change in extent in both the original class at the time of the first observation (T1) and the replacement class in the second observation (T2) (i.e., a land cover **conversion**). More detailed descriptions of the land cover class prior to and following the change can be provided by referencing the categorical and continuous environmental descriptors used in the construction of the FAO LCCS classes at T1 and T2 respectively as well as AEDs. 

Where the Level 3 class remains the same between time-separated periods (the on-diagonals in the transition matrix), a modification rather than conversion of the land cover occurs and only changes in the amounts or type of state indictors within the Level 3 class can take place. These changes can only be described by considering variations (decreases, increases or no difference) in what are termed Essential Environmental Descriptors (EEDs) that are required to construct the FAO LCCS or Additional Environmental Descriptors (AEDs) that are not required but provide additional descriptors.  These descriptors can be units of measurements (or categorizations of these; e.g. canopy cover in %) or simply a change in a pre-defined category (e.g., representing lifeform or leaf type).



## Description 

Through Geoscience Australia's (GA) Digital Earth Australia (DEA) Land Cover, land cover maps have been generated for 34 years (1988 to 2020) from environmental descriptors retreived or classified entirely from Landsat sensor data and according to the Food and Agriculture Organisation (FAO) Land Cover Classification System (Version 2).  The spatial resolution of the mapping is 25 m 
(see DEA Land Cover) at https://cmi.ga.gov.au/data-products/dea/607/dea-land-cover-landsat)

For a selected area of interest, the notebook compares land cover data for any two years and generates maps of conversions (changes in extent) or modifications (no changes in extent but a change in type (e.g. lifeform) or condition of land covers).

1. Loading and displaying DEA land cover classifications (basic and detailed) and contributing environmental descriptors for an area of interest.
2. Determing areas of land cover conversions (losses and gains) based on time-series comparison of the land cover maps between two years.
3. Determining areas of no change in the FAO LCCS Level 3 classes where there is potential for a modification to occur. 

This notebook requires a basic understanding of the DEA Land Cover data set. 
If you are new to DEA Land Cover, it is recommended you look at the introductory [DEA Land Cover notebook](../DEA_datasets/DEA_Land_Cover.ipynb) first. 

This notebook requires a basic understanding of the DEA Land Cover data set. 
If you are new to DEA Land Cover, it is recommended you look at the introductory [DEA Land Cover notebook](../DEA_datasets/DEA_Land_Cover.ipynb) first. 

***

## Getting started

To run this analysis, run all the cells in the notebook starting with the 'Load packages and connect to the datacube' cell.

In [1]:
%matplotlib inline

import datacube
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import pandas as pd
import xarray as xr
import odc.geo.xr
#import rasterio
import geopandas as gpd # Added from WDC

from odc.algo import xr_reproject # From https://docs.dea.ga.gov.au/notebooks/How_to_guides/Reprojecting_data.html

import sys, os
sys.path.insert(1, os.path.abspath('../Tools'))
from dea_tools.plotting import rgb, display_map
from dea_tools.datahandling import wofs_fuser  # Added from DEA plotting
from dea_tools.plotting import rgb, plot_wo  # Added from DEA plotting
from matplotlib import colors as mcolours
from dea_tools.landcover import plot_land_cover, lc_colourmap, make_colorbar
from datacube.testutils.io import rio_slurp_xarray
from datacube.utils import masking # Added from DEA plotting
from datacube.utils.masking import mask_invalid_data
#from datacube.utils import cog  # Temporarily disabled
from datacube.utils.cog import write_cog
# from display_tools import map_geom, rgb # from WDC - not accessing
from ipyleaflet import GeoData # Added from WDC 
from time import time as time # Added from burn notebook
import datetime as dt # Added from burn notebook
from odc.geo.geom import Geometry
from dea_tools.plotting import rgb

from matplotlib.colors import ListedColormap # Added from WDC
import matplotlib.colors as colors # Added from WDC
from matplotlib import colormaps
import matplotlib.pyplot as plt # Added from WDC
from matplotlib.patches import Patch # Added from WDC

import warnings # Added from WDC
warnings.filterwarnings("ignore") # Added from WDC

sys.path.append("../Living_Coasts_training")
from landcover import plot_land_cover, lc_colourmap, make_colorbar #added from DEA

#For DEA burn mapping
from datetime import datetime
from datetime import timedelta

from dea_tools.datahandling import load_ard
from dea_tools.plotting import rgb, display_map
from dea_tools.bandindices import calculate_indices
from dea_tools.dask import create_local_dask_cluster
import display_tools as display_tools

# Create local dask cluster to improve data load time
client = create_local_dask_cluster(return_client=True)

from ipyleaflet import DrawControl, FullScreenControl, LayersControl, Map, Rectangle, WidgetControl, basemaps # Added rectangle RML
from ipywidgets import Button, Dropdown, FloatText, GridspecLayout, Layout, HBox, VBox, Output, Tab, interact, widgets

ImportError: dlopen(/opt/anaconda3/lib/python3.9/site-packages/PIL/_imaging.cpython-39-darwin.so, 0x0002): Library not loaded: @rpath/libtiff.5.dylib
  Referenced from: <1CCD72A1-A76E-3026-9956-EA6FA41A1C72> /opt/anaconda3/lib/python3.9/site-packages/PIL/_imaging.cpython-39-darwin.so
  Reason: tried: '/opt/anaconda3/lib/python3.9/site-packages/PIL/../../../libtiff.5.dylib' (no such file), '/opt/anaconda3/lib/python3.9/site-packages/PIL/../../../libtiff.5.dylib' (no such file), '/opt/anaconda3/bin/../lib/libtiff.5.dylib' (no such file), '/opt/anaconda3/bin/../lib/libtiff.5.dylib' (no such file), '/usr/local/lib/libtiff.5.dylib' (no such file), '/usr/lib/libtiff.5.dylib' (no such file, not in dyld cache)

### Load packages

Load key Python packages and supporting functions for the analysis, then connect to the datacube. 

### Connect to the datacube
Connect to the datacube so we can access DEA data. 

In [ ]:
dc = datacube.Datacube(app="Land_cover_change_mapping")

## Select and view your study area

**If running the notebook for the first time,** keep the default settings below.
This will demonstrate how the change mapping functionality works and provide meaningful results.
The following example loads land cover data over Mount Ney in Western Australia, a region which has experienced extensive bushfires and variations in water inundation over the 34 years of observation.  

**NOTE:  You can change the area of interest and also the time range in subsequent user-selected runs of the notebook.

In [ ]:
# Additional libraries etc.

from datacube.utils.geometry import Geometry, CRS
from ipyleaflet import GeoData
from display_tools import map_geom, rgb

sys.path.append("../Tools/wdc_tools")
from wdc_datahandling import geopolygon_masking

In [ ]:
# add in vector data for area of interest
Area = 'uploads/Fitzroy-riverbend.geojson' 
#Area = 'uploads/AOIs/Fitzroy-mangroves.geojson'
#Area = 'uploads/AOIs/StirlingRanges.geojson'
#Area = 'uploads/AOIs/Stradbroke-Island-north.json'
Area = gpd.read_file(Area)

In [ ]:
display_map(x=(Area.total_bounds[0], Area.total_bounds[2]), y=(Area.total_bounds[1], Area.total_bounds[3]))

In [ ]:
# Set the range of dates for the analysis
time_range = ("1988", "2011")

# Loop through polygons in geodataframe and extract satellite data
for index, row in Area.iterrows(): 
    # Extract the feature's geometry as a datacube geometry object (this needs to be updated eventually for Shapely 2.0)
    geom = Geometry(geom=row.geometry, crs=Area.crs)
#    geom = Geometry(geom=Area.iloc[0].geometry, 
#                         crs=CRS("epsg:32755"))    
    # Update the query to include our geopolygon
    query = {'geopolygon': geom,
    "time": time_range,             
        }

In [ ]:
# Get colour map for image
cmap, norm = lc_colourmap('level3_change_colour_scheme')

# Define dates and/or period of interest


#### Load and reproject CCI Biomass data

## 3.1  Observed change
The observed change considers conversions (i.e., changes in extent of the Level 3 classes) and modification (i.e., the level 3 class remains the same but there are changes in categorical (e.g., lifeform) or continuous (e.g., water hydroperiod) environmental descriptors.  

### Plot broad land cover types over the time-series

## Load and view the FAO LCCS Level 3 data
The following cell will load what are termed the Environmental Descriptors (EDs) used to construct the Basic (Level 3) and subsequently the Detailed (Level 4) land cover maps for the `lat_range`, `lon_range` and `time_range` defined for your area of interest.

You can view each of these layers using the  `plot_land_cover()` function.

** Note:  You can run one, several or all of these to view the appropriate product.

In [ ]:
# Load land cover data for our polygon and time period
#lc_dataset = dc.load(product='lw_landcover_lw', **query)
#lc_dataset_masked = geopolygon_masking(lc_dataset, geopolygon=geom)

# Step 3: Load DEA Land Cover data from the datacube
try:
    lc = dc.load(
        product="ga_ls_landcover_class_cyear_2",
        output_crs="EPSG:32755",
        measurements=[
            "level3",
        ],
        resolution=(-25, 25),
        **query
    )
    print("Data loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Error loading data from the datacube: {e}")
    
lc_dataset_masked = geopolygon_masking(lc, geopolygon=geom)

In [ ]:
# Section 6: Checkbox widgets for selecting datasets
level3_checkbox = widgets.Checkbox(description='Level 3', value=False)

output = widgets.Output()

def plot_selected_datasets():
    with output:
        output.clear_output()
        selected_datasets = []
        if level3_checkbox.value:
            selected_datasets.append('level3')

        for dataset in selected_datasets:
            plt.figure()
            if dataset == 'level3':
                plot_land_cover(lc.level3)
            plt.show()

plot_datasets_button = widgets.Button(description='Plot Selected Data')
plot_datasets_button.on_click(lambda b: plot_selected_datasets())

checkboxes_box = widgets.HBox([
    level3_checkbox
])
display(checkboxes_box)
display(plot_datasets_button)
display(output)

### Level 3 change maps 

A first step towards developing a globally relevant and standardized taxonomy and framework for consistently describing land cover change was to establish the transition matrix between observed broad land cover classes (i.e., OEDs).  This stage was developed and is illustrated using the FAO LCCS given the dichotomous and then hierarchical modular structure of this taxonomy.  Between-class transitions and within-class changes can be identified by comparing these OEDs (i.e., the FAO LCCS Level 3 classes; 8 in total) between any two time-separated periods (i.e., T1 and T2), leading to 64 potential change categories; 56 on the off-diagonals and 8 on the on-diagonals.  

** Note:  There are 6 classes in the case of Australia as there are limited cultivated aquatic landscapes, and artificial and natural water are merged.

In the following steps, you will compare the earliest and latest Level 3 layers in selected time-series.

#### Select the start and end dates for comparisons of enviromental descriptors 
(e.g., Level 3, lifeform, vegetation cover, water state and level 4 (full classification))

#### ALL LEVEL 3 CHANGES

### GAINS

#### Change (gains) in Level 3 from 2010 [0] to 2020 [4 or -1]

In [ ]:
# Select start and end dates for comparison (change to int32 to ensure we can hold number of 6 digits)
start = lc.level3[0].astype(np.int32)
end = lc.level3[-1].astype(np.int32)
ignore_no_change = True
change_vals = (start * 1000) + end
if ignore_no_change:
    change_vals = np.where(start == end, 0, change_vals)

In [ ]:
level_3 = lc.level3[0].drop_vars("time")

In [ ]:
# Create a new Xarray.DataArray
obs_gain_l3_2010_2020 = xr.DataArray(
    data=change_vals,
    coords=level_3.coords,
    dims=level_3.dims,
    name="observed change",
    attrs=level_3.attrs,
    fastpath=False,
)

In [ ]:
obs_gain_l3 = obs_gain_l3_2010_2020.to_dataset(name="l3_2010_2020")
del obs_gain_l3_2010_2020

### Plot Level 3 gains

In [ ]:
# Get colour map for image
#cmap, norm = lc_colourmap('level3_change_colour_scheme')
cmap, norm = lc_colourmap('level3_change_colour_gain')

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd

# Sample code for plotting
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the xarray data without the colorbar
c = obs_gain_l3.l3_2010_2020.plot(ax=ax, cmap=cmap, norm=norm, add_colorbar=False)

# Pretty plot options
ax.margins(0.05)
ax.set_aspect('equal', adjustable='datalim')
ax.set_xlabel('Longitude', fontsize=12)  # Set x-axis label
ax.set_ylabel('Latitude', fontsize=12)  # Set y-axis label
ax.set_title('Level 3 Gains: 2010 to 2020', fontsize=16)

# Create a custom legend
legend_colors = {
    "#FFFFFF": [0.0, "Not classified"],
    "#D1E133": [111.0, "Cultivated or managed terrestrial vegetation"],
    "#007A02": [113.0, "Semi-natural terrestrial woody vegetation"],
    "#95c748": [114.0, "Semi-natural terrestrial herbaceous vegetation"],
    "#4EEEE8": [123.0, "Cultivated or managed aquatic vegetation"],
    "#02C077": [124.0, "Semi-natural aquatic vegetation"],
    "#DA5C69": [215.0, "Artificial surface"],
    "#F3AB69": [216.0, "Bare surface"],
    "#4D9FDC": [220.0, "Water"],
}

legend_labels = [
    "Not classified",
    "Cultivated or managed terrestrial vegetation",
    "Semi-natural terrestrial woody vegetation",
    "Semi-natural terrestrial herbaceous vegetation",
    "Cultivated or managed aquatic vegetation",
    "Semi-natural aquatic vegetation",
    "Artificial surface",
    "Bare surface",
    "Water",
    
]

# Create custom legend
handles = [plt.Line2D([0], [0], marker='o', color='w', label=label, 
                       markerfacecolor=color, markersize=10,
                     markeredgewidth=0.8, markeredgecolor='black') for label, color in zip(legend_labels, legend_colors)]

#ax.legend(handles=handles, loc='lower right', fontsize=10, title='', title_fontsize='13', frameon=True)
ax.legend(handles=handles, loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, title='', title_fontsize='13', frameon=True)

plt.tight_layout()
plt.show()

#obs_gain_l3.l3_2010_2020.plot(cmap=cmap, norm=norm)

In [ ]:
#This method uses the datacube.utils.cog function write_cog (where COG stands for Cloud Optimised GeoTIFF) 
#to export a simple single-band, single time-slice GeoTIFF file. A few important caveats should be noted when using this function:
#It requires an xarray.DataArray; supplying an xarray.Dataset will return an error. 

# Write GeoTIFF to a location
write_cog(geo_im=obs_gain_l3["l3_2010_2020"],
          fname='outputs/obs_gain_l3_2010_2020.tif',
          overwrite=True)

### LOSSES

#### Identify change (loss) in Level 3 from 2010 [0] to 2020 [-1 or 4]

In [ ]:
# Select start and end dates for comparison (change to int32 to ensure we can hold number of 6 digits)
# (Note changing to (end * 1000) + start
start = lc.level3[0].astype(np.int32)
end = lc.level3[-1].astype(np.int32)
ignore_no_change = True
change_vals = (end * 1000) + start
if ignore_no_change:
    change_vals = np.where(start == end, 0, change_vals)
#level_3 = lc.level3[0].drop_vars("time")

In [ ]:
# Create a new Xarray.DataArray
obs_loss_l3_2010_2020= xr.DataArray(
    data=change_vals,
    coords=level_3.coords,
    dims=level_3.dims,
    name="observed losses",
    attrs=level_3.attrs,
    fastpath=False,
)

In [ ]:
obs_loss_l3 = obs_loss_l3_2010_2020.to_dataset(name="l3_2010_2020")
del obs_loss_l3_2010_2020

#### Change from 2017 [1] to 2018 [2]

#### Plot Level 3 losses

In [ ]:
# Get colour map for image
#cmap, norm = lc_colourmap('level3_change_colour_scheme')
cmap, norm = lc_colourmap('level3_change_colour_gain')

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd

# Sample code for plotting
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the xarray data without the colorbar
c = obs_loss_l3.l3_2010_2020.plot(ax=ax, cmap=cmap, norm=norm, add_colorbar=False)

# Pretty plot options
ax.margins(0.05)
ax.set_aspect('equal', adjustable='datalim')
ax.set_xlabel('Longitude', fontsize=12)  # Set x-axis label
ax.set_ylabel('Latitude', fontsize=12)  # Set y-axis label
ax.set_title('Level 3 losses: 2010 to 2020', fontsize=16)

# Create a custom legend
legend_colors = {
    "#FFFFFF": [0.0, "Not classified"],
    "#D1E133": [111.0, "Cultivated or managed terrestrial vegetation"],
    "#007A02": [113.0, "Semi-natural terrestrial woody vegetation"],
    "#95c748": [114.0, "Semi-natural terrestrial herbaceous vegetation"],
    "#4EEEE8": [123.0, "Cultivated or managed aquatic vegetation"],
    "#02C077": [124.0, "Semi-natural aquatic vegetation"],
    "#DA5C69": [215.0, "Artificial surface"],
    "#F3AB69": [216.0, "Bare surface"],
    "#4D9FDC": [220.0, "Water"],
}

legend_labels = [
    "Not classified",
    "Cultivated or managed terrestrial vegetation",
    "Semi-natural terrestrial woody vegetation",
    "Semi-natural terrestrial herbaceous vegetation",
    "Cultivated or managed aquatic vegetation",
    "Semi-natural aquatic vegetation",
    "Artificial surface",
    "Bare surface",
    "Water",
    
]

# Create custom legend
handles = [plt.Line2D([0], [0], marker='o', color='w', label=label, 
                       markerfacecolor=color, markersize=10,
                     markeredgewidth=0.8, markeredgecolor='black') for label, color in zip(legend_labels, legend_colors)]

#ax.legend(handles=handles, loc='lower right', fontsize=10, title='', title_fontsize='13', frameon=True)
ax.legend(handles=handles, loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, title='', title_fontsize='13', frameon=True)

plt.tight_layout()
plt.show()

In [ ]:
#This method uses the datacube.utils.cog function write_cog (where COG stands for Cloud Optimised GeoTIFF) 
#to export a simple single-band, single time-slice GeoTIFF file. A few important caveats should be noted when using this function:
#It requires an xarray.DataArray; supplying an xarray.Dataset will return an error. 

# Write GeoTIFF to a location
write_cog(geo_im=obs_loss_l3["l3_2010_2020"],
          fname='outputs/obs_loss_l3_2010_2020.tif',
          overwrite=True)

### NO CHANGE

#### Areas of no change from 2010 [0] to 2020 [-1 or 4]

In [ ]:
# Select start and end dates for comparison (change to int32 to ensure we can hold number of 6 digits)
start = lc.level3[0].astype(np.int32)
end = lc.level3[-1].astype(np.int32)

# Mark if you want to ignore no change
ignore_no_change = False

# Combine classifications from start and end dates
change_vals = (start * 1000) + end

# Mask out values with no change by setting to 0 if this is requested
if ignore_no_change:
    change_vals = np.where(start == end, 0, change_vals)
    
level_3 = lc.level3[0].drop_vars("time") 

# Create a new Xarray.DataArray
obs_change_l3_2010_2020 = xr.DataArray(
    data=change_vals,
    coords=level_3.coords,
    dims=level_3.dims,
    name="observed change",
    attrs=level_3.attrs,
    fastpath=False,
)

In [ ]:
obs_change_l3 = obs_change_l3_2010_2020.to_dataset(name="l3_2010_2020")
del obs_change_l3_2010_2020

In [ ]:
# Get colour map for image
cmap, norm = lc_colourmap('level3_change_colour_scheme')

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd

# Sample code for plotting
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the xarray data without the colorbar
c = obs_change_l3.l3_2010_2020.plot(ax=ax, cmap=cmap, norm=norm, add_colorbar=False)

# Pretty plot options
ax.margins(0.05)
ax.set_aspect('equal', adjustable='datalim')
ax.set_xlabel('Longitude', fontsize=12)  # Set x-axis label
ax.set_ylabel('Latitude', fontsize=12)  # Set y-axis label
ax.set_title('Level 3 no changes: 2010 to 2020', fontsize=16)

# Create a custom legend
legend_colors = {
    "#FFFFFF": [0.0, "Not classified"],
    "#D1E133": [111.0, "Cultivated or managed terrestrial vegetation"],
    "#007A02": [113.0, "Semi-natural terrestrial woody vegetation"],
    "#95c748": [114.0, "Semi-natural terrestrial herbaceous vegetation"],
    "#4EEEE8": [123.0, "Cultivated or managed aquatic vegetation"],
    "#02C077": [124.0, "Semi-natural aquatic vegetation"],
    "#DA5C69": [215.0, "Artificial surface"],
    "#F3AB69": [216.0, "Bare surface"],
    "#4D9FDC": [220.0, "Water"],
}

legend_labels = [
    "Not classified",
    "Cultivated or managed terrestrial vegetation",
    "Semi-natural terrestrial woody vegetation",
    "Semi-natural terrestrial herbaceous vegetation",
    "Cultivated or managed aquatic vegetation",
    "Semi-natural aquatic vegetation",
    "Artificial surface",
    "Bare surface",
    "Water",
    
]

# Create custom legend
handles = [plt.Line2D([0], [0], marker='o', color='w', label=label, 
                       markerfacecolor=color, markersize=10,
                     markeredgewidth=0.8, markeredgecolor='black') for label, color in zip(legend_labels, legend_colors)]

#ax.legend(handles=handles, loc='lower right', fontsize=10, title='', title_fontsize='13', frameon=True)
ax.legend(handles=handles, loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, title='', title_fontsize='13', frameon=True)

plt.tight_layout()
plt.show()

#### Export as a Geotif

In [ ]:
#This method uses the datacube.utils.cog function write_cog (where COG stands for Cloud Optimised GeoTIFF) 
#to export a simple single-band, single time-slice GeoTIFF file. A few important caveats should be noted when using this function:
#It requires an xarray.DataArray; supplying an xarray.Dataset will return an error. 

# Write GeoTIFF to a location
write_cog(geo_im=obs_change_l3["l3_2010_2020"],
          fname='outputs/no_change_l3_2010_2020.tif',
          overwrite=True)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** February 2022

**Compatible datacube version:** 

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)